### Loading Library

In [65]:
import cv2
import tensorflow as tf
import pathlib
from tensorflow.keras.models import load_model
import os

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

### Load Test Images

In [67]:
#load our model 
model = load_model(os.path.join(r'C:\\Users\\16676\\Downloads\\early-detection-of-3d-printing-issues\\Models\\','hasunderextrusion_or_not_model_12.h5'))
test_images_path=(r'C:\\Users\\16676\\Downloads\\early-detection-of-3d-printing-issues\\Input\\Final_test\\')


In [68]:

valid_datagen= ImageDataGenerator(rescale=1./255)
test_generator = valid_datagen.flow_from_directory(
    directory=test_images_path,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)


Found 25279 images belonging to 1 classes.


### Predict Classification of the Images

In [48]:
# Predict from generator (returns probabilities)
pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

C:\Users\16676\AppData\Local\Temp/ipykernel_12224/3387772560.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred=model.predict_generator(test_generator, steps=len(test_generator), verbose=1)


25279/25279 [==============================] - 1263s 50ms/step


In [49]:
# Get classes by np.round
cl = np.round(pred)
# Get filenames (set shuffle=false in generator is important)
filenames=test_generator.filenames

In [50]:
# Data frame
results=pd.DataFrame({"Test_file":filenames,"predication":pred[:,0], "has_under_extrusion":cl[:,0].astype(int)})

In [51]:
results['ID']=results['Test_file'].apply(lambda x:str(x).split("\\")[1])
col_change=results.pop('ID')
results.insert(0, 'ID', col_change)

In [52]:
results.head(2)

,ID,Test_file,predication,has_under_extrusion
0,1672776120.647672.jpg,test_images\1672776120.647672.jpg,5.769526e-08,0
1,1672776121.654801.jpg,test_images\1672776121.654801.jpg,6.079748e-09,0


In [53]:
len(results)

25279

In [56]:
df_Sample_Sumbission=pd.read_csv(r'C:\\Users\\16676\\Downloads\\early-detection-of-3d-printing-issues\\Input\\test.csv')

In [57]:
df_Sample_Sumbission['ID']=df_Sample_Sumbission['img_path'].apply(lambda x:str(x).split('/')[2])
col_change=df_Sample_Sumbission.pop('ID')
df_Sample_Sumbission.insert(0, 'ID', col_change)

In [58]:
df_Sample_Sumbission.head(2)

,ID,img_path,printer_id,print_id
0,1678578538.704966.jpg,101/1678578332/1678578538.704966.jpg,101,1678578332
1,1678578539.108019.jpg,101/1678578332/1678578539.108019.jpg,101,1678578332


In [59]:
len(df_Sample_Sumbission)

25279

In [60]:
# df_Sample_Sumbission.to_csv('test_real.csv')

In [61]:
#merging the imagies with their extrusion types
df_Sample_Sumbission=df_Sample_Sumbission.merge(results ,how='left')

In [62]:
df_Sample_Sumbission.head(2)

,ID,img_path,printer_id,print_id,Test_file,predication,has_under_extrusion
0,1678578538.704966.jpg,101/1678578332/1678578538.704966.jpg,101,1678578332,test_images\1678578538.704966.jpg,1.0,1
1,1678578539.108019.jpg,101/1678578332/1678578539.108019.jpg,101,1678578332,test_images\1678578539.108019.jpg,1.0,1


In [63]:
df_Sample_Sumbission = df_Sample_Sumbission.drop(columns=['ID','printer_id','print_id','Test_file','predication'])


In [64]:
df_Sample_Sumbission.to_csv(r'C:\\Users\\16676\\Downloads\\early-detection-of-3d-printing-issues\\Data\\sample_submission_v12.csv',index=False)